# 継承と仮想関数


## キーポイント

* 継承は、あるクラスを元に、別の新しいクラスを定義する機能
* 継承は次のように書く
    ```cpp
    class 新しいクラス名 : アクセス指定子 元になるクラス名 {
      新しいクラスのメンバ
    };
    ```
* 継承の元になるクラスを「基底クラス」、継承で定義される新しいクラスを「派生クラス」という
* 「アクセス指定子」には通常は`public`を選ぶ(`protected`と`private`は`is-a`関係にならないため)
* 基底クラスのメンバ関数に`virtual`(バーチャル)キーワードを付けると「仮想関数」になる
    ```cpp
    virtual void メンバ関数名() {}
    ```
* 派生クラスのメンバ関数に`override`(オーバーライド)キーワードを付けると、基底クラスの仮想関数を上書きできる
    ```cpp
    void メンバ関数名() override {}
    ```
* 継承を使うときは、基底クラスのデストラクタを「仮想デストラクタ」にする


----

## 1 継承(けいしょう)

----

### 1.1 継承の動作

継承(けいしょう)は、「ある構造体を元に、メンバを追加した別の構造体を定義する」機能です。

ある構造体を継承した別の構造体を定義するには、`:`(コロン)記号を使って次のように書きます。

```cpp
struct 新しく作る構造体名 : 元になる構造体名 {
};
```

継承を使ったとき、元になる構造体は「基底(きてい)クラス」と呼ばれます。<br>
新しく定義する構造体は「派生(はせい)クラス」と呼ばれます。

```cpp
struct 派生クラス : 基底クラス {
};
```

次のプログラムは、`Character`(キャラクター)構造体を継承して、`Fighter`(ファイター)構造体と`Mage`(メイジ)構造体を定義する例です。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

// キャラクターの基底クラス
struct Character
{
  void ShowNameAndHP() { cout << name << " 体力:" << hp << '/' << hp_max; }

  string name;
  int hp_max, hp;
};

// 戦士のキャラクター
struct Fighter : Character
{
  void ShowStatus() { cout << "剣の威力:" << sword_power << " 鎧:" << armor; }

  int sword_power;
  int armor;
};

// 魔術師のキャラクター
struct Mage : Character
{
  void ShowStatus() { cout << "魔法の威力:" << magic_power << "魔力:" << mp << '/' << mp_max; }

  int magic_power;
  int mp_max, mp;
};

int main() {
  Fighter laios = { "ライオス", 20, 15, 10, 8 };
  Mage marcille = { "マルシル", 8, 8, 20, 10, 10 };

  laios.ShowNameAndHP();
  cout << ' ';
  laios.ShowStatus();

  marcille.ShowNameAndHP();
  cout << ' ';
  marcille.ShowStatus();
}
```

**実行結果**

```txt
ライオス 体力:15/20 剣の威力:10 鎧:8
マルシル 体力:8/8 魔法の威力: 20 魔力:10/10
```

この例では、`Character`構造体が「基底クラス」、`Figher`構造体と`Mage`構造体が「派生クラス」に当たります。


### 1.2 継承とコンストラクタ

基底クラスにコンストラクタがある場合、以下の例のように、派生クラスのコンストラクタの初期化リストの最初に、基底クラスのコンストラクタを書きます。

```cpp
struct Character
{
  Character(string n, int h) : name(n), hp_max(h), hp(h) {}

  string name;
  int hp_max, hp;
};

struct Fighter : Character
{
  //                                       ↓基底クラスのコンストラクタ
  Fighter(string n, int h, int s, int a) : Character(n, h), sword_power(s), armor(a) {}

  int sword_power;
  int armor;
};
```


### 1.3 継承のアクセス制御

構造体を継承するとき、「アクセス指定子」を追加できます。<br>
アクセス指定子には、`public`(パブリック)、`private`(プライベート)、`protected`(プロテクテッド)の3種類があります。

```cpp
struct B {};
//         ↓アクセス指定子
struct D : public B {};
```

継承は、基底クラスのメンバを派生クラスに持ち込みます。<br>
アクセス指定子を使うと、基底クラスのメンバを持ち込むときに、メンバのアクセス指定子を上書きできます。<br>
どのアクセス指定子が何を上書きするかは、次の表を参考にしてください。

| アクセス指定子 | 基底クラスの<br>パブリックメンバ | 基底クラスの<br>プロテクテッドメンバ | 基底クラスの<br>ポインタへの代入 |
|:----:|:----:|:---:|:---:|
| <font size=3>public</font><br>(パブリック) | publicのまま | protectedのまま | ◯ |
| <font size=3>protected</font><br>(プロテクテッド) | protectedになる | protectedのまま | ✕ |
| <font size=3>private</font><br>(プライベート) | privateになる | privateになる | ✕ |

>基底クラスのプライベートメンバは、どのアクセス指定子を選んだ場合でも`private`のままです。

クラスの継承を行う場合、基本的には`public`アクセス指定子が使われます。他の2種類は滅多に使われません。<br>
`protected`や`private`アクセス指定子を選ぶと「基底クラスのポインタへの代入」ができなくなるからです。

また、アクセス指定子を指定しなかった場合、派生クラスが構造体なら`public`指定子、クラスなら`private`指定子を設定したものとして扱われます。

```cpp
struct B {};     // 基底クラスがstructかclassかは無関係
struct S : B {}; // public継承
class C : B {};  // private継承
```


#### 多重継承

元になる構造体を２つ以上指定することもできます。これは「多重継承(たじゅうけいしょう)」と呼ばれます。<br>
多重継承を使うと、複数の基底クラスを持つ派生クラスを作れます。

```cpp
struct A {};
class B {};
class C : public A, public B {}; // 多重継承
```

多重継承は主に「インターフェイス・クラス」という概念とともに利用されます。<br>
少し高度な話題になるため詳しい説明は省略しますが、興味がある場合はWebで検索したり、参考書で調べてみるとよいでしょう。



### 1.4 継承と包含

ところで、「ある構造体を元にする」だけなら、「構造体変数を別の構造体のメンバ変数にする」、つまり「包含(ほうがん)」でも実現できます。例えば、上記のプログラムは包含を使って次のように書くこともできます。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

// キャラクターの共通データ
struct Character
{
  void ShowNameAndHP() { cout << name << " 体力:" << hp << '/' << hp_max; }

  string name;
  int hp_max, hp;
};

// 戦士のキャラクター
struct Fighter
{
  void ShowNameAndHP() { charecter.ShowNameAndHP(); } // 包含した型の関数を呼び出す
  void ShowStatus() { cout << "剣の威力:" << sword_power << " 鎧:" << armor; }

  Character character; // Characterを包含
  int sword_power;
  int armor;
};

// 魔術師のキャラクター
struct Mage
{
  void ShowNameAndHP() { charecter.ShowNameAndHP(); } // 包含した型の関数を呼び出す
  void ShowStatus() { cout << "魔法の威力:" << magic_power << "魔力:" << mp << '/' << mp_max; }

  Character character; // Characterを包含
  int magic_power;
  int mp_max, mp;
};

int main() {
  Fighter laios = { "ライオス", 20, 15, 10, 8 };
  Mage marcille = { "マルシル", 8, 8, 20, 10, 10 };

  laios.ShowNameAndHP();
  cout << ' ';
  laios.ShowStatus();

  marcille.ShowNameAndHP();
  cout << ' ';
  marcille.ShowStatus();
}
```

**実行結果**

```txt
ライオス 体力:15/20 剣の威力:10 鎧:8
マルシル 体力:8/8 魔法の威力: 20 魔力:10/10
```

このように、「構造体を元に別の構造体を定義する」ことは、継承を使わなくても可能なのです。<br>
それなのに継承という機能が存在するのは、継承には次の2つの機能があるからです。

* 「仮想関数(かそうかんすう)」という、名前が同じで内容だけが派生クラスごとに異なるメンバ関数を作れる
* 基底クラスの参照(またはポインタ)を、派生クラスの参照(またはポインタ)で初期化できる(ポインタの場合は代入もできる)


### 1.5 is-a関係とhas-a関係

継承と包含のどちらを使うべきかを決める指標として、`is-a`(イズ・ア)関係と`has-a`(ハズ・ア)関係があります。


#### is-a関係

`is-a`関係とは`A is a B`、つまり「AはBの一種です」という関係で、例えば次のようなものです:

* 戦士はキャラクターの一種です(戦士 is a キャラクター)
* 軽トラは自動車の一種です(軽トラ is a 自動車)

このような`is-a`関係は「継承(けいしょう)」であらわします。

>なお、`is-a`関係を作れるのは`public`な継承だけです。`protected`継承や`private`継承は`has-a`関係になります。


#### has-a関係

`has-a`関係とは`A has a B`、つまり「AはBを持っています」という関係です。

* 戦士は武器を持っています(戦士 has a 武器)
* 自動車にはエンジンが載っています(自動車 has a エンジン)

このような`has-a`関係は「包含(ほうがん)」であらわします。


#### 継承と包含の選びかた

基本的な考えかたは、

>「AはBの一種です」と言える場合に限って継承を使う

というものです。<br>
それ以外、つまり「AはBの一種ではない」という場合は包含を使います。

例えば、「戦士はキャラクター(人物)の一種です」とは言えても「戦士はキャラクター(人物)を持っています」とは言いにくいです。<br>
そのため、継承が適切だと考えられます。

別の例として、軽トラとエンジンの関係を考えてみましょう。<br>
「軽トラはエンジンを持っています」とは言いますが「軽トラはエンジンの一種です」とはあまり言いません。<br>
そのため、包含が適切だと考えられます。

要点は、「プログラムの部品同士の関係を適切に表しているか」という観点で評価することです。<br>
「その言い方はありうるか」は考慮しません。

>例えば、キャラクターという単語を「性格」という意味でとらえれば、「戦士はキャラクター(性格)を持っています」と言うこともできるでしょう。ただし、この言い方が成立するのは、プログラムにおいて「キャラクター構造体が実際に性格を表している場合」に限られます。


----

## 2 仮想関数

----


### 2.1 仮想関数

仮想関数(かそうかんすう)とは、

>基底クラスの参照(またはポインタ)から仮想関数を呼び出すと、派生クラスで上書きした関数が実行される

という機能です。仮想関数は、次の手順で作成します。

1. 「基底クラス」のメンバ関数を「仮想関数として宣言」する。
2. 1で宣言した仮想関数を「派生クラス」で「上書き」する。

メンバ関数を仮想関数として宣言するには、`virtual`(バーチャル)キーワードを使って次のように書きます。

```cpp
struct 基底クラス {
//↓virtualキーワードを付けると仮想関数になる
  virtual 戻り型 仮想関数名(引数1, ...) { プログラム }
};
```

仮想関数を上書きするには、`override`(オーバーライド)キーワードを使って次のように書きます。

```cpp
struct 派生クラス : 基底クラス {
//                              ↓overrideキーワードを付けると仮想関数を上書きできる
  戻り型 仮想関数名(引数1, ...) override { プログラム }
};
```

また、仮想関数が`const`メンバ関数だった場合、`const`キーワードの後ろに`override`キーワードを書きます。

```cpp
struct 基底クラス {
  virtual 戻り型 仮想関数名(引数1, ...) const { プログラム }
};

struct 派生クラス : 基底クラス {
  戻り型 仮想関数名(引数1, ...) const override { プログラム }
};
```

このように、基底クラスの仮想関数を派生クラスで上書きすることを「オーバーライド」といいます。

次のプログラムは、仮想関数とオーバーライドを使った例です。

```cpp
#include <iostream>
#include <string>
using namespace std;

// 基底クラス
struct Base {
  // メンバ関数 Func を「仮想関数」として宣言
  virtual void Func(const string& s) const { cout << s << endl; }
};

// 派生クラス
struct D : public Base {
  // メンバ関数 Func をオーバーライド(上書き)
  void Func(const string& s) const override {
    cout << string(s.size() + 4, '#') << endl;
    cout << "# " << s << " #" << endl;
    cout << string(s.size() + 4, '#') << endl;
  }
};

int main() {
  // Base型のオブジェクトをBase型のポインタに代入
  shared_ptr<Base> a = make_shared<Base>();
  a->Func("Base"); // Base::Funcが呼び出される

  // D型のオブジェクトをBase型のポインタに代入
  shared_ptr<Base> b = make_shared<D>();
  b->Func("Derived"); // D::Funcが呼び出される
}
```

**実行結果**

```txt
Base
###########
# Derived #
###########
```

普通のメンバ関数と仮想関数では、呼び出す関数を選択するルールが異なります。<br>
普通のメンバ関数の選択ルールはシンプルで、「ポインタ(または参照)の型のメンバ関数」が呼び出されます。

対して、仮想関数の選択ルールはやや複雑です。<br>
「呼び出そうとしているメンバ関数」が仮想関数だった場合、まずポインタ(または参照)が指している「オブジェクトの型」を調べます。そして、オブジェクトの型が「呼び出そうとしているメンバ関数」をオーバーライドしている場合、オーバーライドしたメンバ関数が呼び出されます。

<img width="450px" hspace="50px" src="https://raw.githubusercontent.com/tn-mai/cpp2025/refs/heads/main/images/virtual_function_and_override.png" />

> よく分からない場合は、とりあえず「派生クラスでオーバーライドした関数が呼ばれる」ことだけ覚えてください。


#### オーバーライドするときの注意点

仮想関数をオーバーライドするときは、以下のすべてが上書きする仮想関数と一致していなくてはなりません。<br>
少しでも違っていると、コンパイルエラーになります。

* 戻り型
* 仮想関数名
* 引数


### 2.2 メンバ関数と仮想関数の違い

仮想関数を使わない場合、基底クラスと派生クラスで同じ名前のメンバ関数を定義しても、それらは別々の関数として扱われます。そして、基底クラスの参照からは基底クラスのメンバ関数が呼び出され、派生クラスの参照からは派生クラスのメンバ関数が呼び出されます。

次のプログラムは、仮想関数を使わない場合の例です。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Base {
   void f() { cout << "Baseのfメンバ関数を実行" << endl; }
};

struct Sub : Base {
   void f() { cout << "Subのfメンバ関数を実行" << endl; }
};

void g(A& a) { a.f(); }

int main() {
  Sub sub;
  sub.f();

  Base& base = sub;
  base.f();
}
```

**実行結果**

```txt
Subのfメンバ関数を実行
Baseのfメンバ関数を実行
```

参照変数`base`は変数`sub`を参照していますが、`sub`は実際には`Sub`構造体の変数です。しかし、`base.f()`という関数呼び出しでは`Baseのfメンバ関数を実行`という文章が出力されます。これは、C++のルールで「呼び出す関数は変数の型で決まる」と決まっているからです。

次に、`f`メンバ関数を仮想関数にしたプログラムを見てみましょう。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Base {
   virtual void f() { cout << "Baseのfメンバ関数を実行" << endl; }
};

struct Sub : Base {
   void f() override { cout << "Subのfメンバ関数を実行" << endl; }
};

void g(A& a) { a.f(); }

int main() {
  Sub sub;
  sub.f();

  Base& base = sub;
  base.f();
}
```

**実行結果**

```txt
Subのfメンバ関数を実行
Subのfメンバ関数を実行
```

このプログラムでも、参照変数`base`は変数`sub`を参照しており、`sub`は`Sub`構造体の変数という部分は同じです。ですが、`base.f()`が出力するのは`Subのfメンバ関数を実行`という文章に変わっています。

このように、仮想関数を呼び出すと「プログラム実行時に参照している変数の、本来の型で定義された関数」が実行されます。


### 2.3 仮想関数の例

構造体の種類に応じて処理を分けたい、というのはよくある問題です。例えば、ゲームに戦士と魔術師の2種類のユニットが登場するとします。彼らは構造体も異なるでしょうし、その行動プログラムも違ったものになるでしょう。

C++には構造体の種類を比較する機能がないので、ユニットの種類を見分けるには、構造体に種類を示すメンバ変数を追加するのが一般的です。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

struct Character {
  Character(int t, string n, int hm, int h) : type(t), name(n), hp_max(hm), hp(h) {}

  int type; // 1=戦士 2=魔術師
  string name;
  int hp_max, hp;
};

struct Fighter : Character {
  Fighter(string n, int hm, int h, int p, int a) : Character(1, n, hm, h), sword_power(p), armor(a) {}

  void Attack() { cout << "剣で攻撃" << endl; }

  int sword_power;
  int armor;
};

struct Mage : Character {
  Mage(string n, int hm, int h, int mm, int m, int p) : Character(2, n, hm, h), mp_max(mm), mp(m), magic_power(p) {}

  void Attack() { cout << "魔法で攻撃" << endl; }

  int mp_max, mp;
  int magic_power;
};

int main() {
  Fighter laios = { "ライオス", 20, 20, 10, 8 };
  Mage marcille = { "マルシル", 8, 8, 10, 10, 20 };
  vector<Character*> v = { &laios, &marcille };

  for (int i = 0; i < v.size(); i++) {
    // ユニットの種類によって呼び出す関数を変える
    if (v[i]->type == 1) {
      Fighter* p = (Fighter*)v[i];
      p->Attack();
    } else if (v[i]->type == 2) {
      Mage* p = (Mage*)v[i];
      p->Attack();
    }
  }
}
```

**実行結果**

```txt
剣で攻撃
魔法で攻撃
```

このプログラムでは、`Character`構造体に、種類をあらわす`type`(タイプ)というメンバ変数を追加しました。

そして、`for`文では`type`メンバ変数の値を見て、呼び出すべきメンバ関数を切り替えています。

このプログラムの問題は、ユニットの種類を増やすたびに、「呼び出す関数を変える部分にもプログラムを追加しなくてはならない」ことです。もし追加し忘れると、そのユニットは行動できなくなってしまいます。

しかし、仮想関数を使えば、「追加し忘れる」という問題を解決できます。<br>
次のプログラムは、先ほどのプログラムを仮想関数を使って書き換えたものです。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

struct Character {
  Character(string n, int hm, int h) : name(n), hp_max(hm), hp(h) {}

  virtual void Attack() { cout << "(未設定)" << endl; }; // 仮想関数

  string name;
  int hp_max, hp;
};

struct Fighter : Character {
  Fighter(string n, int hm, int h, int p, int a) : Character(n, hm, h), sword_power(p), armor(a) {}

  void Attack() override { cout << "剣で攻撃" << endl; } // 仮想関数を上書き

  int sword_power;
  int armor;
};

struct Mage : Character {
  Mage(string n, int hm, int h, int mm, int m, int p) : Character(n, hm, h), mp_max(mm), mp(m), magic_power(p) {}

  void Attack() override { cout << "魔法で攻撃" << endl; } // 仮想関数を上書き

  int mp_max, mp;
  int magic_power;
};

int main() {
  Fighter laios = { "ライオス", 20, 20, 10, 8 };
  Mage marcille = { "マルシル", 8, 8, 10, 10, 20 };
  vector<Character*> v = { &laios, &marcille };

  for (int i = 0; i < v.size(); i++) {
    // ユニットの種類によって呼び出す関数を変える
    v[i]->Attack(); // 仮想関数を呼び出す
  }
}
```

**実行結果**

```txt
剣で攻撃
魔法で攻撃
```

「種類によって呼び出す関数を変える」プログラムが、たったの1行に減っていることに注目してください。にもかかわらず、実行結果は以前と同じです。

これは、基底クラスのポインタから仮想関数を呼び出すと「仮想関数を上書きしたメンバ関数」が実行される、という機能のおかげです。

例えば、`v[0]`の型は`Character*`ですが、代入した`laious`変数の本来の型は`Fighter`構造体です。<br>
そして、`Fighter`構造体は`Attack`仮想関数を上書きしています。<br>
その結果、`v[0]->Attack()`は、「`Character`構造体の`Attack`仮想関数」ではなく、「`Fighter`構造体の`Attack`メンバ関数」を実行します。

同様に`v[1]`には`marcille`変数のアドレスが代入されています。<br>
`marcille`は`Mage`構造体の変数で、`Mage`構造体は`Attack`仮想関数を上書きしています。<br>
そのため、`v[1]->Attack()`では、「`v[1]`が参照している`marcille`変数の、本来の型である`Mage`構造体の、`Attack`メンバ関数」が実行されるのです。

>**【継承と仮想関数の習得には時間が必要】**<br>
>この節のプログラムは、「継承、配列、ポインタを組わせたときにできること」の例になっています。<br>
>特に、仮想関数を活用するには、ポインタをうまく使うことが重要となります。
>
>とはいえ、仮想関数の考え方は少し難しいです。継承と仮想関数を十分に理解するには、それなりの時間がかかるでしょう。<br>
>当面、「仮想関数を使わない例のほうが分かりやすい」と感じた場合は、無理に仮想関数を使う必要はありません。<br>
>そのうえで、仮想関数を少しずつ試して、徐々に使い方に慣れていってください。


### 2.4 仮想デストラクタ

継承を使うと決めた場合、基底クラスのデストラクタを「仮想デストラクタ」にしなくてはなりません。

基底クラスのポインタや参照からメンバ関数を呼び出すと、通常は基底クラスのメンバ関数が実行されます。この動作はデストラクタであっても変わりません。

例えば、`new`よって作成した派生クラスの変数を、基底クラスのポインタや参照に代入して制御しているとします。<br>
このとき、ポインタや参照を`delete`すると、基底クラスのデストラクタが実行されます。

次のプログラムは、「デストラクタを仮想関数にしなかった」場合の例です。

**コード**

```cpp
struct Base {
  ~Base() { cout << "destruct Base" << endl; }
};

struct D : Base {
  ~D() { cout << "destruct D" << endl; }
  string s;
};

int main() {
  Base* p = new D;
  delete p; // Baseのデストラクタが呼ばれる
}
```

**実行結果**

```txt
destruct Base
```

この例では、`new`で作成したのは`D`構造体のはずなのに、`D`のデストラクタが呼び出されていません(`destruct D`が表示されていない)。<br>
その結果、`D`のメンバ変数`s`は適切に破棄されず、メモリリークなどの危険な状態になってしまいます。

`delete`において、本来実行されるべき「派生クラスのデストラクタを呼び出す」には、基底クラスのデストラクタを仮想デストラクタにします。

デストラクタを仮想デストラクタにするには、次のように、デストラクタの定義に`virtual`(バーチャル)キーワードを付けます。<br>
そして、派生クラスのデストラクタには`override`キーワードを付けます。

```cpp
struct Base {
  // 仮想デストラクタ
  virtual ~Base() { cout << "destruct Base" << endl; }
};
```

次のプログラムは、「デストラクタを仮想関数にした」場合の例です。

**コード**

```cpp
struct Base {
  virtual ~Base() { cout << "destruct Base" << endl; }
};

struct D : Base {
  ~D() override { cout << "destruct D" << endl; }
  string s;
};

int main() {
  Base* p = new D;
  delete p; // DとBaseの両方のデストラクタが呼ばれる
}
```

**実行結果**

```txt
destruct D
destruct Base
```

このように、派生クラスのデストラクタを仮想デストラクタにしておけば、派生クラスを適切に破棄できます。

>厳密には、継承を使っていても仮想デストラクタにしなくてよい場合もあります。<br>
>ですが、少し難しい問題なので、今のところは「クラスを継承するときは必ず仮想デストラクタにする」と覚えてください。

また、仮想デストラクタは、次の点で通常の仮想関数とは動作が異なります。

* 仮想デストラクタでは、派生クラスのデストラクタだけでなく、基底クラスのデストラクタも実行される
* 基底クラスが仮想でストラタを持つ場合、派生クラスのデストラクタは自動的に仮想デストラクタになる<br>(明示的に定義しない場合も含む)

通常の仮想関数では派生クラスの関数だけが実行されます。<br>
しかし、仮想デストラクタの場合は基底クラスのデストラクタも実行されるのです。<br>
上記のプログラムの実行結果で、`destruct D`の下に`destruct Base`という文が表示されるのは、この仕組みがあるからです。

デストラクタだけ特別扱いになっている理由は、

>基底クラスのデストラクタを呼び出さないと、基底クラスを適切に破棄できない

ためです。基底クラスの破棄方法は派生クラスには分からないので、このような仕組みになっています。


### 2.5 純粋仮想関数(じゅんすい・かそうかんすう)

仮想関数を作るとき、オーバーライドすることが前提で、基底クラスをそのまま使うことは想定していない、という場合があります。<br>
確実にオーバーライドさせたい場合は、仮想関数を「純粋仮想関数」として宣言します。

メンバ関数を純粋仮想関数として宣言するには、`virtual`キーワードに加えて、宣言の末尾に`= 0`を付け加えます。

次のプログラムは、純粋仮想関数を宣言する例です。

```cpp
struct Base {
// ↓ virtualキーワードを付けると仮想関数になる
   virtual void f() = 0;
//                  ↑ 仮想関数に`= 0`を追加すると純粋仮想関数になる
};
```

クラスにひとつでも純粋仮想関数があると、そのクラスのオブジェクトは作成できなくなります。<br>
その結果、派生クラスを作成し、純粋仮想関数をオーバーライドすることが強制されます。

純粋仮想関数を持ち、そのままでは作成できないクラスのことを「抽象(ちゅうしょう)クラス」といいます。<br>
また、基底クラスのすべての純粋仮想関数を実装し、作成可能になった派生クラスのことを「具象(ぐしょう)クラス」といいます。

次のプログラムは、抽象クラスと具象クラスの例です。

```cpp
#include <iostreawm>
#include <string>
#include <vector>
#include <memory>
using namespace std;

// 抽象クラス
struct Enemy {
   Enemy(const string& s) : name(s) {}
   virtual ~Enemy() = default; // 仮想デストラクタ

   virtual void Attack() = 0; // 純粋仮想関数
   void Dead() { cout << name << "は死んだ" << endl; } // 普通のメンバ関数

   string name;
};

// 具象クラス
struct Goblin : Enemy {
  Goblin() : Enemy("ゴブリン") {}
  ~Goblin() override = default;

  void Attack() override { cout << "ゴブリンは棍棒で殴りかかった！" << endl; }
};

// これも具象クラス
struct Slime : Enemy {
  Slime() : Enemy("スライム") {}
  ~Slime() override = default;
  
  void Attack() override { cout << "スライムは毒の触手を伸ばした！" << endl; }
};

int main() {
  vector<shared_ptr<Enemy>> v[3];
  v[0] = make_shared<Goblin>();
  v[1] = make_shared<Slime>();
  v[2] = make_shared<Goblin>();
  for (int i = 0; i < (int)v.size(); i++) {
    v[i]->Attack();
  }
}
```

この例の`Enemy`構造体は、「何らかの攻撃手段を持つ敵」として定義されています。<br>
そして、具体的な攻撃手段は派生クラスで定義します。<br>

そして、プログラムでは、`Enemy`の純粋仮想関数だけを使って敵を操作します。実際の敵の種類は気にしません。<br>
このように、抽象クラスは具象クラスに対する「インターフェイス」として機能します。


----

## 3 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ 基底ゾンビと派生ゾンビ

ゾンビ構造体があります。ゾンビ構造体の派生クラスとして、「高速移動ゾンビ」と「ひっかきゾンビ」を作ることにしました。<br>
作成済みの「高速移動ゾンビ」構造体を参考にして、「ひっかきゾンビ」構造体を作成してください。

>**高速移動ゾンビ構造体(状態:完成)**
>
>* 構造体名: DashZombie
>    * `Zombie`構造体を継承
>* メンバ変数:
>    * `dash_speed`: `int`型、移動速度
>* メンバ関数:
>    * 名前: Dash
>    * 戻り値: なし
>    * 引数: なし
>    * 機能:
>        1. `cout`に`Zombie::GetName()`の戻り値を出力
>        2. `cout`に、文字列`"の高速移動(速度="`、メンバ変数`dash_speed`の値、文字列`")"`を出力<br>(記号は半角英数で入力すること)

>**ひっかきゾンビ構造体(状態:未定義)**
>
>* 構造体名: ScratchZombie
>    * `Zombie`構造体を継承
>* メンバ変数:
>    * `clow_power`: `int`型、ひっかき攻撃力
>* メンバ関数:
>    * 名前: Scratch
>    * 戻り値: なし
>    * 引数: なし
>    * 機能:
>        1. `cout`に`Zombie::GetName()`の戻り値を出力
>        2. `cout`に、文字列`"のひっかき攻撃(威力="`、メンバ変数`clow_power`の値、文字列`")"`を出力<br>(記号は半角英数で入力すること)

**入力データ形式**

```txt
名前 噛みつき攻撃力 移動速度
名前 噛みつき攻撃力 ひっかき攻撃力
```

**入力例**

```txt
ボブ 8 5
アリス 4 7
```

**出力例**

```txt
ゾンビ化したボブの高速移動(速度=5)
ゾンビ化したボブの噛みつき攻撃(威力=8)
ゾンビ化したアリスのひっかき攻撃(威力=7)
ゾンビ化したアリスの噛みつき攻撃(威力=4)
```


In [ ]:
%%writefile practice_01a.cpp
#include <string>
#include <iostream>
using namespace std;

// ゾンビの基底クラス
struct Zombie
{
  string GetName() { return "ゾンビ化した" + name; }
  void Bite() { cout << GetName() << "の噛みつき攻撃(威力:" << jaw_power << ")" << endl; }

  string name;
  int jaw_power;
};

// 高速移動ゾンビ構造体
struct DashZombie : Zombie
{
  void Dash() { cout << GetName() << "の高速移動(速度=" << dash_speed << ")" << endl; }

  int dash_speed;
};
// --- ここから上は変更しない ---

// この下に、「ひっかきゾンビ」構造体の定義を書く


// --- ここから下は変更しない ---
int main() {
  string a;
  int b, c;
  cin >> a >> b >> c;

  string d;
  int e, f;
  cin >> d >> e >> f;

  DashZombie dz = { a, b, c };
  ScratchZombie sz = { d, e, f };

  dz.Dash();
  dz.Bite();

  sz.Scratch();
  sz.Bite();
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、高速移動ゾンビとひっかきゾンビのデータを入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "ゾンビ化したボブの高速移動(速度=5)\nゾンビ化したボブの噛みつき攻撃(威力=8)\nゾンビ化したアリスのひっかき攻撃(威力=7)\nゾンビ化したアリスの噛みつき攻撃(威力=4)\nゾンビ化したdogの高速移動(速度=10)\nゾンビ化したdogの噛みつき攻撃(威力=10)\nゾンビ化したcatのひっかき攻撃(威力=10)\nゾンビ化したcatの噛みつき攻撃(威力=5)") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "ボブ 8 5 アリス 4 7" | ./practice_01a && echo "dog 10 10 cat 5 10" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <string>
#include <iostream>
using namespace std;

// ゾンビの基底クラス
struct Zombie
{
  string GetName() { return "ゾンビ化した" + name; }
  void Bite() { cout << GetName() << "の噛みつき攻撃(威力=" << jaw_power << ")" << endl; }

  string name;
  int jaw_power;
};

// 高速移動ゾンビ構造体
struct DashZombie : Zombie
{
  void Dash() { cout << GetName() << "の高速移動(速度=" << dash_speed << ")" << endl; }

  int dash_speed;
};
// --- ここから上は変更しない ---

// この下に、「ひっかきゾンビ」構造体の定義を書く

// ひっかきゾンビ構造体
struct ScratchZombie : Zombie
{
  void Scratch() { cout << GetName() << "のひっかき攻撃(威力=" << claw_power << ")" << endl; }

  int claw_power;
};

// --- ここから下は変更しない ---
int main() {
  string a;
  int b, c;
  cin >> a >> b >> c;

  string d;
  int e, f;
  cin >> d >> e >> f;

  DashZombie dz = { a, b, c };
  ScratchZombie sz = { d, e, f };

  dz.Dash();
  dz.Bite();

  sz.Scratch();
  sz.Bite();
}

### ❓️問題２ 戦士と魔術師はキャラクターの一種です

キャラクター構造体があります。キャラクター構造体の派生クラスとして、「戦士」と「魔術師」を作ることにしました。<br>
作成済みの「戦士」の構造体を参考にして、「魔術師」の構造体を作成してください。

**戦士の構造体**

* 構造体名: Fighter(ファイター)
    * `Character`構造体を継承
* メンバ変数:
    * `sword_power`: `int`型、攻撃力
    * `armor`: `int`型、防御力
* コンストラクタ
    * 引数:
        * `n`: `string`型、名前
        * `h`: `int`型、最大HPおよび現在HP
        * `s`: `int`型、攻撃力
        * `a`: `int`型、防御力
    * 機能:
        * 初期化子リストで`Character`コンストラクタを呼び出す
        * 攻撃力を引数`s`で、守備力を引数`a`で初期化する
* メンバ関数:
    * 名前: ShowStatus(ショウ・ステータス)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        1. 引数に文字列`"戦士"`を指定して、`ShowBaseStatus()`を呼び出す
        2. `cout`に、文字列`",攻撃力="`、メンバ変数`sword_power`の値、文字列`",防御力="`、メンバ変数`armor`の値、の順で出力<br>(記号は半角英数で入力すること)

**魔術師の構造体**

* 構造体名: Mage(メイジ)
    * `Character`構造体を継承
* メンバ変数:
    * `max_mp`: `int`型、最大MP
    * `mp`: `int`型、現在のMP
* コンストラクタ
    * 引数:
        * `n`: `string`型、名前
        * `h`: `int`型、最大HPおよび現在HP
        * `m`: `int`型、最大MPおよび現在MP
    * 機能:
        * 初期化子リストで`Character`コンストラクタを呼び出す
        * 最大MPと現在MPを引数`m`で初期化する
* メンバ関数:
    * 名前: ShowStatus(ショウ・ステータス)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        1. 引数に文字列`"魔術師"`を指定して、`ShowBaseStatus()`を呼び出す
        2. `cout`に、文字列`",MP="`、メンバ変数`mp`の値、文字列`"/"`、メンバ変数`max_mp`の値、の順で出力<br>(英数および記号は半角英数で入力すること)

**入力データ形式**

```txt
戦士の名前 HP 攻撃力 防御力
魔術師の名前 HP MP
```

**入力例**

```txt
キャラモン 57 18 11
レイストリン 13 108
```

**出力例**

```txt
キャラモン(戦士),HP=57/57,攻撃力=18,防御力=11
レイストリン(魔術師),HP=13/13,MP=108/108
```


In [ ]:
%%writefile practice_01b.cpp
#include <string>
#include <iostream>
using namespace std;

// キャラクターの基底クラス
struct Character
{
public:
  // コンストラクタ
  Character(const string& n, int h) : name(n), max_hp(h), hp(h) {}

  // キャラクターの基本情報を出力するメンバ関数
  void ShowBaseStatus(const string& job) {
    cout << name << "(" << job << "),HP=" << hp << "/" << max_hp;
  }

private:
  string name; // 名前
  int max_hp;  // 最大HP
  int hp;      // 現在のHP
};

// 戦士のキャラクター
struct Fighter : Character
{
public:
  // コンストラクタ
  Fighter(string n, int h, int s, int a) : Character(n, h), sword_power(s), armor(a) {}

  // 戦士キャラクターの情報を出力するメンバ関数
  void ShowStatus() {
    ShowBaseStatus("戦士");
    cout << ",攻撃力=" << sword_power << ",防御力=" << armor;
  }

private:
  int sword_power; // 攻撃力
  int armor;       // 防御力
};
// --- ここから上は変更しない ---

// この下に、「魔術師」構造体の定義を書く


// --- ここから下は変更しない ---
int main() {
  string a;
  int b, c, d;
  cin >> a >> b >> c >> d;

  string e;
  int f, g;
  cin >> e >> f >> g;

  Fighter fighter(a, b, c, d);
  Mage mage(e, f, g);

  fighter.ShowStatus();
  cout << endl;

  mage.ShowStatus();
  cout << endl;
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "この下をクリックして、戦士と魔術師のデータを入力" && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "キャラモン(戦士),HP=57/57,攻撃力=18,防御力=11\nレイストリン(魔術師),HP=13/13,MP=108/108\nアラゴルン(戦士),HP=43/43,攻撃力=21,防御力=17\nガンダルフ(魔術師),HP=65/65,MP=255/255") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && echo "キャラモン 57 18 11 レイストリン 13 108" | ./practice_01b && echo "アラゴルン 43 21 17 ガンダルフ 65 255" | ./practice_01b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include <string>
#include <iostream>
using namespace std;

// キャラクターの基底クラス
struct Character
{
public:
  // コンストラクタ
  Character(const string& n, int h) : name(n), max_hp(h), hp(h) {}

  // キャラクターの基本情報を出力するメンバ関数
  void ShowBaseStatus(const string& job) {
    cout << name << "(" << job << "),HP=" << hp << "/" << max_hp;
  }

private:
  string name; // 名前
  int max_hp;  // 最大HP
  int hp;      // 現在のHP
};

// 戦士のキャラクター
struct Fighter : Character
{
public:
  // コンストラクタ
  Fighter(string n, int h, int s, int a) : Character(n, h), sword_power(s), armor(a) {}

  // 戦士キャラクターの情報を出力するメンバ関数
  void ShowStatus() {
    ShowBaseStatus("戦士");
    cout << ",攻撃力=" << sword_power << ",防御力=" << armor;
  }

private:
  int sword_power; // 剣の攻撃力
  int armor;       // 防御力
};
// --- ここから上は変更しない ---

// この下に、「魔術師」構造体の定義を書く
// 魔術師のキャラクター
struct Mage : Character
{
public:
  // コンストラクタ
  Mage(string n, int h, int m) : Character(n, h), max_mp(m), mp(m) {}

  // 魔術師キャラクターの情報を出力するメンバ関数
  void ShowStatus() {
    ShowBaseStatus("魔術師");
    cout << ",MP=" << mp << "/" << max_mp;
  }

private:
  int max_mp; // 最大MP
  int mp;     // 現在のMP
};

// --- ここから下は変更しない ---
int main() {
  string a;
  int b, c, d;
  cin >> a >> b >> c >> d;

  string e;
  int f, g;
  cin >> e >> f >> g;

  Fighter fighter(a, b, c, d);
  Mage mage(e, f, g);

  fighter.ShowStatus();
  cout << endl;

  mage.ShowStatus();
  cout << endl;
}

### ❓️問題３ 動物の鳴き声

動物ごとに異なる鳴き声を出すプログラムを作っています。<br>
基底クラスとして`Animal`(アニマル)構造体を作り、派生クラスとして`Dog`(ドッグ)構造体を作りました。<br>

`Dog`構造体を参考にして、猫と鳥とバッタの派生クラスを追加しなさい。

なお、入力は動物の種類をあらわす`D`, `C`, `B`, `L`の4つの大文字の列Sとして与えられます。<br>
`DCBL`以外の文字はすべて`Animal`として扱います。

| 文字 | 種類 |
|:----:|:----:|
| D | 犬 |
| C | 猫 |
| B | 鳥 |
| L | バッタ |
| 上記以外 | 動物 |

**犬の構造体**

* 構造体名: Dog(ドッグ)
    * `Animal`構造体を継承
* メンバ変数: なし
* コンストラクタ: なし(コンパイラに任せる)
* メンバ関数:
    * 名前: MakeSound(メイク・サウンド)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        * `Animal::MakeSound`をオーバーライドする
        * `cout`に文字列`"ワン"`を出力する

**猫の構造体**

* 構造体名: Cat(キャット)
    * `Animal`構造体を継承
* メンバ変数: なし
* コンストラクタ: なし(コンパイラに任せる)
* メンバ関数:
    * 名前: MakeSound(メイク・サウンド)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        * `Animal::MakeSound`をオーバーライドする
        * `cout`に文字列`"ニャー"`を出力する

**鳥の構造体**

* 構造体名: Bird(バード)
    * `Animal`構造体を継承
* メンバ変数: なし
* コンストラクタ: なし(コンパイラに任せる)
* メンバ関数:
    * 名前: MakeSound(メイク・サウンド)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        * `Animal::MakeSound`をオーバーライドする
        * `cout`に文字列`"オエー"`を出力する

**バッタの構造体**

* 構造体名: Locust(ローカスト)
    * `Animal`構造体を継承
* メンバ変数: なし
* コンストラクタ: なし(コンパイラに任せる)
* メンバ関数:
    * 名前: MakeSound(メイク・サウンド)
    * 戻り値: なし
    * 引数: なし
    * 機能:
        * `Animal::MakeSound`をオーバーライドする
        * `cout`に文字列`"チキチキ"`を出力する

**入力例**

```txt
DCBLA
```

**出力例**

```txt
ワンニャーオエーチキチキガオー
```


In [ ]:
%%writefile practice_01c.cpp
#include <iostream>
#include <string>
#include <vector>
#include <memory>
using namespace std;

// 動物の基底クラス
struct Animal {
  virtual ~Animal() = default;
  virtual string MakeSound() const { return "ガオー"; }
};

// 犬の構造体
struct Dog : Animal {
  ~Dog() override = default;
  string MakeSound() const override { return "ワン"; }
};
// --- ここから上は変更しない ---

// この下に、猫の構造体を定義する


// この下に、鳥の構造体を定義する


// この下に、バッタの構造体を定義する


// --- ここから下は変更しない ---
int main() {
  string s;
  cin >> s;

  vector<shared_ptr<Animal>> v(s.size());
  for (int i = 0; i < (int)s.size(); i++) {
    switch (s[i]) {
      case 'D': v[i] = make_shared<Dog>(); break;
      case 'C': v[i] = make_shared<Cat>(); break;
      case 'B': v[i] = make_shared<Bird>(); break;
      case 'L': v[i] = make_shared<Locust>(); break;
      default:  v[i] = make_shared<Animal>(); break;
    }
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << (*i)->MakeSound();
  }
  cout << endl;
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "この下をクリックして、動物の種類をあらわす文字列を入力" && ./practice_01c

In [ ]:
# @title 実行
!diff -Z <(echo -e "ワンニャーオエーチキチキガオー\nガオーオエーニャーワンガオーガオーガオーガオーガオーガオーガオーチキチキガオーガオーガオーガオーガオーガオーガオーガオーガオーガオーガオーガオーガオーガオー") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01c practice_01c.cpp && echo "DCBLA" | ./practice_01c && echo "ABCDEFGHIJKLMNOPQRSTUVWXYZ" | ./practice_01c) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01c.cpp
#include <iostream>
#include <string>
#include <vector>
#include <memory>
using namespace std;

// 動物の基底クラス
struct Animal {
  virtual ~Animal() = default;
  virtual string MakeSound() const { return "ガオー"; }
};

// 犬の構造体
struct Dog : Animal {
  ~Dog() override = default;
  string MakeSound() const override { return "ワン"; }
};
// --- ここから上は変更しない ---

// この下に、猫の構造体を定義する
// 猫の構造体
struct Cat : Animal {
  ~Cat() override = default;
  string MakeSound() const override { return "ニャー"; }
};

// この下に、鳥の構造体を定義する
// 鳥の構造体
struct Bird : Animal {
  ~Bird() override = default;
  string MakeSound() const override { return "オエー"; }
};

// この下に、バッタの構造体を定義する
// バッタの構造体
struct Locust : Animal {
  ~Locust() override = default;
  string MakeSound() const override { return "チキチキ"; }
};

// --- ここから下は変更しない ---
int main() {
  string s;
  cin >> s;

  vector<shared_ptr<Animal>> v(s.size());
  for (int i = 0; i < (int)s.size(); i++) {
    switch (s[i]) {
      case 'D': v[i] = make_shared<Dog>(); break;
      case 'C': v[i] = make_shared<Cat>(); break;
      case 'B': v[i] = make_shared<Bird>(); break;
      case 'L': v[i] = make_shared<Locust>(); break;
      default:  v[i] = make_shared<Animal>(); break;
    }
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << (*i)->MakeSound();
  }
  cout << endl;
}

### ❓️問題４ お絵かき

6x3の画面に、点、線、正方形の図形を描くプログラムがあります。<br>
`Point`構造体と`Box`構造体の定義を参考に、以下の仕様を満たす「線をあらわす`Line`構造体」を定義しなさい。

>画面の何も描かれていない部分(空白)は`.`であらわされます。

**図形の共通仕様**

* `Shape`(シェイプ)構造体を継承する
* `Draw`仮想関数をオーバーライドして、独自の図形を描く
* 図形を描くには`Canvas`(キャンバス)クラスの`Put`(プット)メンバ関数を使う

**図形：点の仕様**

* 構造体名は`Point`(ポイント)とする
* `Draw`メンバ関数で、座標(X, Y)の位置に`*`(アスタリスク)記号を描く

**図形：線の仕様**

* 構造体名は`Line`(ライン)とする
* `Draw`メンバ関数で、座標(X, Y)および座標(X+1, Y)の位置に`-`(マイナス)記号を描く

**図形：正方形の仕様**

  * 構造体名は`Box`(ボックス)とする
  * `Draw`メンバ関数で、座標(X, Y)、(X+1,Y)、(X,Y+1)、(X+1, Y+1)の4つの位置に`#`(シャープ)記号を描く

**入力データ形式**

```txt
点の数A 線の数B 正方形の数C
点1のX座標 点1のY座標 点2のX座標 点2のY座標 … 点AのX座標 点AのY座標
線1のX座標 線1のY座標 線2のX座標 線2のY座標 … 線AのX座標 線AのY座標
正方形1のX座標 正方形1のY座標 正方形2のX座標 正方形2のY座標 … 正方形CのX座標 正方形CのY座標
```

**入力データ例（１）**

```txt
1 1 1
4 0
0 2
2 1
```

**出力例（１）**

```txt
....*.
..++..
--++..
```

**入力データ例（２）**

```txt
2 4 1
1 2 4 2
1 1 -1 2 3 1 5 2
2 0
```

**出力例（２）**

```txt
..++..
.-++-.
-*..*-
```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <vector>
using namespace std;

// 描画キャンバス
class Canvas {
public:
  Canvas(int w, int h) : image(w * h, '.'), width(w), height(h) {}

  // x,y座標の位置に文字cを描く
  void Put(int x, int y, char c) {
    if (x >= 0 && x < width && y >= 0 && y <= height) {
      image[y * width + x] = c;
    }
  }

  void Print() {
    for (int y = 0; y < height; y++) {
      for (int x = 0; x < width; x++) {
        cout << image[y * width + x];
      }
      cout << endl;
    }
  }

private:
  vector<char> image;
  int width, height;
};

// 図形の基底クラス
struct Shape {
  virtual ~Shape() = default;
  virtual void Draw(Canvas&) {}
  int x, y;
};

// 図形：点
struct Point : Shape {
  ~Point() override = default;
  void Draw(Canvas& canvas) override {
    canvas.Put(x, y, '*');
  }
};
// --- ここから上は変更しない ---

// この下に、「図形：線の仕様」を満たす構造体を定義する


// --- ここから下は変更しない ---
// 図形：正方形
struct Box : Shape {
  ~Box() override = default;
  void Draw(Canvas& canvas) override {
    canvas.Put(x, y, '+');
    canvas.Put(x + 1, y, '+');
    canvas.Put(x, y + 1, '+');
    canvas.Put(x + 1, y + 1, '+');
  }
};

int main() {
  int a, b, c;
  cin >> a >> b >> c;

  vector<Shape*> shapes;

  vector<Point> points(a);
  for (int i = 0; i < a; i++) {
    cin >> points[i].x >> points[i].y;
    shapes.push_back(&points[i]);
  }

  vector<Line> lines(b);
  for (int i = 0; i < b; i++) {
    cin >> lines[i].x >> lines[i].y;
    shapes.push_back(&lines[i]);
  }

  vector<Box> boxes(c);
  for (int i = 0; i < c; i++) {
    cin >> boxes[i].x >> boxes[i].y;
    shapes.push_back(&boxes[i]);
  }

  Canvas canvas(6, 3);
  for (int i = 0; i < a + b + c; i++) {
    shapes[i]->Draw(canvas);
  }
  canvas.Print();
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、3つの図形の数、点の座標、線の座標、四角形の座標を入力:" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "....*.\n..++..\n--++..\n..++..\n.-++-.\n-*..*-") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "1 1 1 4 0 0 2 2 1" | ./practice_02a && echo "2 4 1 1 2 4 2 1 1 -1 2 3 1 5 2 2 0" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒️解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <vector>
using namespace std;

// 描画キャンバス
class Canvas {
public:
  Canvas(int w, int h) : image(w * h, '.'), width(w), height(h) {}

  // x,y座標の位置に文字cを描く
  void Put(int x, int y, char c) {
    if (x >= 0 && x < width && y >= 0 && y <= height) {
      image[y * width + x] = c;
    }
  }

  void Print() {
    for (int y = 0; y < height; y++) {
      for (int x = 0; x < width; x++) {
        cout << image[y * width + x];
      }
      cout << endl;
    }
  }

private:
  vector<char> image;
  int width, height;
};

// 図形の基底クラス
struct Shape {
  virtual ~Shape() = default;
  virtual void Draw(Canvas&) {}
  int x, y;
};

// 図形：点
struct Point : Shape {
  ~Point() override = default;
  void Draw(Canvas& canvas) override {
    canvas.Put(x, y, '*');
  }
};
// --- ここから上は変更しない ---

// この下に、「図形：線の仕様」を満たす構造体を定義する
struct Line : Shape {
  ~Line() override = default;
  void Draw(Canvas& canvas) override {
    canvas.Put(x, y, '-');
    canvas.Put(x + 1, y, '-');
  }
};

// --- ここから下は変更しない ---
// 図形：正方形
struct Box : Shape {
  ~Box() override = default;
  void Draw(Canvas& canvas) override {
    canvas.Put(x, y, '+');
    canvas.Put(x + 1, y, '+');
    canvas.Put(x, y + 1, '+');
    canvas.Put(x + 1, y + 1, '+');
  }
};

int main() {
  int a, b, c;
  cin >> a >> b >> c;

  vector<Shape*> shapes;

  vector<Point> points(a);
  for (int i = 0; i < a; i++) {
    cin >> points[i].x >> points[i].y;
    shapes.push_back(&points[i]);
  }

  vector<Line> lines(b);
  for (int i = 0; i < b; i++) {
    cin >> lines[i].x >> lines[i].y;
    shapes.push_back(&lines[i]);
  }

  vector<Box> boxes(c);
  for (int i = 0; i < c; i++) {
    cin >> boxes[i].x >> boxes[i].y;
    shapes.push_back(&boxes[i]);
  }

  Canvas canvas(6, 3);
  for (int i = 0; i < a + b + c; i++) {
    shapes[i]->Draw(canvas);
  }
  canvas.Print();
}

### ❓️問題５ 迷路と敵

迷路内の敵について、配置パターンの番号sと、移動回数nを指定すると、n回移動したあとの敵の位置を出力するプログラムがあります。

敵の種類は4つで、以下のプログラムでは、左右に移動する敵`EnemyMoveX`(エネミー・ムーブ・エックス)と、右の壁に沿って進む敵`EnemyAlongRightWall`(エネミー・アロング・ライト・ウォール)の2種類が完成しています。

完成している2種類の敵の定義とプログラム中のコメントを参考にして、以下の仕様を満たすように、残りの2つの敵構造体を定義しなさい。

>この問題の動作テストは、敵の動きを視覚的に確認できるようになっています。うまく活用してください。<br>
>&emsp;赤: 左右に移動する敵<br>
>&emsp;緑: 上下に移動する敵<br>
>&emsp;黄: 右の壁に沿って移動する敵<br>
>&emsp;青: 左の壁に沿って移動する敵

**敵の共通仕様**

* `Enemy`基底クラスを継承する
* `Move`仮想関数をオーバーライドして、独自の動きかたをプログラムする

**左右に移動する敵の仕様(状態:完成)**

* 構造体名: EnemyMoveX(エネミー・ムーブ・エックス)
* 機能:
    * 最初は「左」を向いている(コンストラクタの引数`d`に`3`を指定)
    * 前方が壁の場合は後ろに振り向く
    * 前方が壁以外の場合は一歩前進する

**上下に移動する敵の仕様(状態:未定義)**

* 構造体名: EnemyMoveY(エネミー・ムーブ・ワイ)
* 機能:
    * 最初は「上」を向いている(コンストラクタの引数`d`に`0`を指定)
    * 前方が壁の場合は後ろに振り向く
    * 前方が壁以外の場合は一歩前進する

**右の壁に沿って移動する敵の仕様(状態:完成)**

* 構造体名: EnemyAlongRightWall(エネミー・アロング・ライト・ウォール)
* 機能:
    * 最初は「右」を向いている(コンストラクタの引数`d`に`1`を指定)
    * 左側が壁以外の場合は左を向き、一歩前進する
    * 左側が壁の場合、かつ前方も壁の場合は右を向く
    * 左側が壁の場合で、前方が壁以外の場合は一歩前進する

**左の壁に沿って移動する敵の仕様(状態:未定義)**

* 構造体名: EnemyAlongLeftWall(エネミー・アロング・レフト・ウォール)
* 機能:
    * 最初は「下」を向いている(コンストラクタの引数`d`に`2`を指定)
    * 左側が壁以外の場合は左を向き、一歩前進する
    * 左側が壁の場合、かつ前方も壁の場合は右を向く
    * 左側が壁の場合で、前方が壁以外の場合は一歩前進する

**入力データ例（１）**

```txt
1 7
```

**出力例（１）**

```txt
10 3 11 5 2 7 9 3
```

**入力データ例（１）**

```txt
813 10
```

**出力例（１）**

```txt
11 3 3 3 14 3 15 2
```


In [ ]:
%%writefile practice_02b.cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

// 迷路をあらわすクラス
class Maze {
public:
  Maze() {
    data = { "#################",
             "#...#...#.....#.#",
             "#.#.###.#.###.#.#",
             "#.#...#.....#...#",
             "###.#.###.#.#####",
             "#...#.#...#...#.#",
             "#.###.#.#.#.#.#.#",
             "#.......#...#...#",
             "#################" };
    width = data[0].size();
    height = data.size();
  }

  // 座標(x, y)の地形を取得(#=壁 .=通路)
  char Get(int x, int y) { return data[y][x]; }

  // 通路上のランダムな位置を取得
  int GetRandomX() { return Rand() % (width / 2) * 2 + 1; }
  int GetRandomY() { return Rand() % (height / 2) * 2 + 1; }

  // 迷路の幅と高さを取得
  int GetWidth() { return width; }
  int GetHeight() { return height; }

  // 疑似乱数の初期値を設定
  void SetRand(int n) { random_seed = n; }

  // 疑似乱数を作成
  int Rand() {
    random_seed = random_seed * 48271;
    return int(random_seed / 4096);
  }

private:
  vector<string> data;
  int width, height;
   unsigned int random_seed = 1; // 疑似乱数データ
};

// 敵の基底クラス
struct Enemy {
  Enemy(int x, int y, int d) : x(x), y(y), dir(d) {}
  virtual ~Enemy() {}

  // 移動を扱う仮想関数
  virtual void Move(Maze&) {}

  // 向いている方向に1マス進む
  void Walk() { x += dx[dir]; y += dy[dir]; }

  // 右を向く
  void TurnRight() { dir = (dir + 1) % 4; }

  // 後ろを向く
  void TurnAround() { dir = (dir + 2) % 4; }

  // 左を向く
  void TurnLeft() { dir = (dir + 3) % 4; }

  // true=前方に壁がある false=前方には壁はない
  bool IsWallFront(Maze& maze) { return IsWall(maze, dir); }

  // true=右側に壁がある false=右側には壁はない
  bool IsWallRight(Maze& maze) { return IsWall(maze, (dir + 1) % 4); }

  // true=左側に壁がある false=左側には壁はない
  bool IsWallLeft(Maze& maze) { return IsWall(maze, (dir + 3) % 4); }

  // true=方向dに壁がある false=方向dには壁はない
  bool IsWall(Maze& maze, int d) { return maze.Get(x + dx[d], y + dy[d]) == '#'; }

  int x, y; // 位置
  int dir;  // 方向(0=上 1=右 2=下 3=左)
  vector<int> dx = { 0, 1, 0, -1 }; // 向きに対応するX方向の移動値
  vector<int> dy = { -1, 0, 1, 0 }; // 向きに対応するY方向の移動値
};

// 左右に移動する敵
struct EnemyMoveX : Enemy {
  EnemyMoveX(int x, int y) : Enemy(x, y, 3) {}

  void Move(Maze& maze) override {
    if (IsWallFront(maze)) {
      TurnAround();
    } else {
      Walk();
    }
  }
};

// 右の壁に沿って進む敵
struct EnemyAlongRightWall : Enemy {
  EnemyAlongRightWall(int x, int y) : Enemy(x, y, 1) {}

  void Move(Maze& maze) override {
    if ( ! IsWallRight(maze)) {
      TurnRight();
      Walk();
    } else if (IsWallFront(maze)) {
      TurnLeft();
    } else {
      Walk();
    }
  }
};
// --- ここから上は変更しない ---

// この下に、「上下に移動する敵の仕様」を満たす構造体を定義する


// この下に、「左の壁に沿って進む敵の仕様」を満たす構造体を定義する


// --- ここから下は変更しない ---
int main() {
  int s, n;
  cin >> s >> n;

  Maze maze;
  maze.SetRand(s);

  EnemyMoveX ex = { maze.GetRandomX(), maze.GetRandomY() };
  EnemyMoveY ey = { maze.GetRandomX(), maze.GetRandomY()};
  EnemyAlongRightWall er = { maze.GetRandomX(), maze.GetRandomY() };
  EnemyAlongLeftWall el = { maze.GetRandomX(), maze.GetRandomY() };
  vector<Enemy*> enemies = { &ex, &ey, &er, &el };

#ifndef SHOW_STEP
  for (int a = 0; a < n; a++) {
    for (int b = 0; b < (int)enemies.size(); b++) {
      enemies[b]->Move(maze);
    }
  }
#else
  // 動作テスト用のプログラム
  int w = maze.GetWidth(), h = maze.GetHeight();
  for (int a = 0; ; ) {
    cout << "turn=" << a + 1 << endl;
    for (int y = 0; y < h; y++) {
      for (int x = 0; x < w; x++) {
        bool hasEnemy = false;
        for (int i = 0; i < (int)enemies.size(); i++) {
          if (enemies[i]->x == x && enemies[i]->y == y) {
            cout << "\033[3" << char('1' + i) << ";1m";
            const char c[] = { 'A', '>', 'V', '<' };
            cout << c[enemies[i]->dir];
            cout << "\033[0;1m";
            hasEnemy = true;
            break;
          }
        }
        if (!hasEnemy) {
          cout << maze.Get(x, y);
        }
      }
      cout << endl;
    }
    cout << endl;
    if (++a > n) {
      break;
    }
    for (int b = 0; b < (int)enemies.size(); b++) {
      enemies[b]->Move(maze);
    }
  }
#endif // SHOW_STEP

  for (int i = 0; i < (int)enemies.size(); i++) {
    cout << enemies[i]->x << ' ' << enemies[i]->y << ' ';
  }
  cout << endl;
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -D SHOW_STEP -o practice_02b practice_02b.cpp && echo "この下をクリックして、配置パターン番号と移動回数を入力:" && ./practice_02b

In [ ]:
# @title 実行
!diff -Z <(echo -e "10 3 11 5 2 7 1 7\n11 3 3 3 14 3 15 2") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02b practice_02b.cpp && echo "1 7" | ./practice_02b && echo "813 10" | ./practice_02b) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒️解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02b.cpp
#include <iostream>
#include <vector>
#include <string>
using namespace std;

// 迷路をあらわすクラス
class Maze {
public:
  Maze() {
    data = { "#################",
             "#...#...#.....#.#",
             "#.#.###.#.###.#.#",
             "#.#...#.....#...#",
             "###.#.###.#.#####",
             "#...#.#...#...#.#",
             "#.###.#.#.#.#.#.#",
             "#.......#...#...#",
             "#################" };
    width = data[0].size();
    height = data.size();
  }

  // 座標(x, y)の地形を取得(#=壁 .=通路)
  char Get(int x, int y) { return data[y][x]; }

  // 通路上のランダムな位置を取得
  int GetRandomX() { return Rand() % (width / 2) * 2 + 1; }
  int GetRandomY() { return Rand() % (height / 2) * 2 + 1; }

  // 迷路の幅と高さを取得
  int GetWidth() { return width; }
  int GetHeight() { return height; }

  // 疑似乱数の初期値を設定
  void SetRand(int n) { random_seed = n; }

  // 疑似乱数を作成
  int Rand() {
    random_seed = random_seed * 48271;
    return int(random_seed / 4096);
  }

private:
  vector<string> data;
  int width, height;
   unsigned int random_seed = 1; // 疑似乱数データ
};

// 敵の基底クラス
struct Enemy {
  Enemy(int x, int y, int d) : x(x), y(y), dir(d) {}
  virtual ~Enemy() {}

  // 移動を扱う仮想関数
  virtual void Move(Maze&) {}

  // 向いている方向に1マス進む
  void Walk() { x += dx[dir]; y += dy[dir]; }

  // 右を向く
  void TurnRight() { dir = (dir + 1) % 4; }

  // 後ろを向く
  void TurnAround() { dir = (dir + 2) % 4; }

  // 左を向く
  void TurnLeft() { dir = (dir + 3) % 4; }

  // true=前方に壁がある false=前方には壁はない
  bool IsWallFront(Maze& maze) { return IsWall(maze, dir); }

  // true=右側に壁がある false=右側には壁はない
  bool IsWallRight(Maze& maze) { return IsWall(maze, (dir + 1) % 4); }

  // true=左側に壁がある false=左側には壁はない
  bool IsWallLeft(Maze& maze) { return IsWall(maze, (dir + 3) % 4); }

  // true=方向dに壁がある false=方向dには壁はない
  bool IsWall(Maze& maze, int d) { return maze.Get(x + dx[d], y + dy[d]) == '#'; }

  int x, y; // 位置
  int dir;  // 方向(0=上 1=右 2=下 3=左)
  vector<int> dx = { 0, 1, 0, -1 }; // 向きに対応するX方向の移動値
  vector<int> dy = { -1, 0, 1, 0 }; // 向きに対応するY方向の移動値
};

// 左右に移動する敵
struct EnemyMoveX : Enemy {
  EnemyMoveX(int x, int y) : Enemy(x, y, 3) {}

  void Move(Maze& maze) override {
    if (IsWallFront(maze)) {
      TurnAround();
    } else {
      Walk();
    }
  }
};

// 右の壁に沿って進む敵
struct EnemyAlongRightWall : Enemy {
  EnemyAlongRightWall(int x, int y) : Enemy(x, y, 1) {}

  void Move(Maze& maze) override {
    if ( ! IsWallRight(maze)) {
      TurnRight();
      Walk();
    } else if (IsWallFront(maze)) {
      TurnLeft();
    } else {
      Walk();
    }
  }
};
// --- ここから上は変更しない ---

// この下に、「上下に移動する敵の仕様」を満たす構造体を定義する
struct EnemyMoveY : Enemy {
  EnemyMoveY(int x, int y) : Enemy(x, y, 0) {}

  void Move(Maze& maze) override {
    if (IsWallFront(maze)) {
      TurnAround();
    } else {
      Walk();
    }
  }
};

// この下に、「左の壁に沿って進む敵の仕様」を満たす構造体を定義する
struct EnemyAlongLeftWall : Enemy {
  EnemyAlongLeftWall(int x, int y) : Enemy(x, y, 2) {}

  void Move(Maze& maze) override {
    if ( ! IsWallLeft(maze)) {
      TurnLeft();
      Walk();
    } else if (IsWallFront(maze)) {
      TurnRight();
    } else {
      Walk();
    }
  }
};

// --- ここから下は変更しない ---
int main() {
  int s, n;
  cin >> s >> n;

  Maze maze;
  maze.SetRand(s);

  EnemyMoveX ex = { maze.GetRandomX(), maze.GetRandomY() };
  EnemyMoveY ey = { maze.GetRandomX(), maze.GetRandomY()};
  EnemyAlongRightWall er = { maze.GetRandomX(), maze.GetRandomY() };
  EnemyAlongLeftWall el = { maze.GetRandomX(), maze.GetRandomY() };
  vector<Enemy*> enemies = { &ex, &ey, &er, &el };

#ifndef SHOW_STEP
  for (int a = 0; a < n; a++) {
    for (int b = 0; b < (int)enemies.size(); b++) {
      enemies[b]->Move(maze);
    }
  }
#else
  // 動作テスト用のプログラム
  int w = maze.GetWidth(), h = maze.GetHeight();
  for (int a = 0; ; ) {
    cout << "turn=" << a + 1 << endl;
    for (int y = 0; y < h; y++) {
      for (int x = 0; x < w; x++) {
        bool hasEnemy = false;
        for (int i = 0; i < (int)enemies.size(); i++) {
          if (enemies[i]->x == x && enemies[i]->y == y) {
            cout << "\033[3" << char('1' + i) << ";1m";
            const char c[] = { 'A', '>', 'V', '<' };
            cout << c[enemies[i]->dir];
            cout << "\033[0;1m";
            hasEnemy = true;
            break;
          }
        }
        if (!hasEnemy) {
          cout << maze.Get(x, y);
        }
      }
      cout << endl;
    }
    cout << endl;
    if (++a > n) {
      break;
    }
    for (int b = 0; b < (int)enemies.size(); b++) {
      enemies[b]->Move(maze);
    }
  }
#endif // SHOW_STEP

  for (int i = 0; i < (int)enemies.size(); i++) {
    cout << enemies[i]->x << ' ' << enemies[i]->y << ' ';
  }
  cout << endl;
}